<a href="https://colab.research.google.com/github/JHyunjun/torch_Generative-Adversarial-Network/blob/main/torch_PROJECT_TSGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import array
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#Data Load
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/train_v.txt') 
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/test_v.txt')
train_data_input = df_train.iloc[:,:-1]
train_data_output = df_train.iloc[:,-1]
test_data_input = df_test.iloc[:,:-1]
test_data_output = df_test.iloc[:,-1]

train_data = df_train.iloc[:,:]
test_data = df_test.iloc[:,:]

trainSet_for_result = train_data
testSet_for_result = test_data

print("train data input : ",train_data_input.shape,train_data_input)
print("train data output : ",train_data_output.shape, train_data_output)
print("train_data : ",train_data.shape,train_data)
print("test_data : ",test_data.shape,test_data)

In [ ]:
#train_gan = df_train.iloc[0:len(df_train),2]
train_gan = df_train.iloc[0:600, 2]
print(train_gan.shape)
print(train_gan)
#print(len(train_gan))

train_gan2 = train_gan
#train_gan2 = df_train.iloc[200:400, -1]
train_gan2 = train_gan2.to_numpy()
print(train_gan2.shape)

In [ ]:
#Normalization
def MinMaxScaler(data):
  denom = np.max(data,0) - np.min(data,0)
  #denom = 750 - 450
  nume = data-np.min(data,0)
  #nume = data - 450
  return nume/denom

def back_MinMax(data,max,min):
  #max = 750
  #min = 450
  diff = max - min
  back = data * diff + min
  return back

#train_gan_norm = MinMaxScaler(train_gan)
print(train_gan.shape)
train_gan_max = np.max(train_gan)
train_gan_min = np.min(train_gan)
train_gan = MinMaxScaler(train_gan)
train_gan = torch.tensor(train_gan)
#train_gan_flatten = torch.flatten(train_gan)
#train_gan_flatten = train_gan_flatten.double().type()
train_gan = torch.tensor(train_gan, dtype = torch.float32)
train_gan_flatten = train_gan
print(train_gan_flatten)

print("train_gan_max : ", train_gan_max)
print("train_gan_min : ", train_gan_min)

In [ ]:
#train_gan_norm = MinMaxScaler(train_gan)
print(train_gan2.shape)
train_gan2_max = np.max(train_gan2)
train_gan2_min = np.min(train_gan2)
train_gan2 = MinMaxScaler(train_gan2)
train_gan2 = torch.tensor(train_gan2)
#train_gan_flatten = torch.flatten(train_gan)
#train_gan_flatten = train_gan_flatten.double().type()
train_gan2 = torch.tensor(train_gan2, dtype = torch.float32)
train_gan2_flatten = train_gan2
print(train_gan2_flatten)

In [ ]:
Data_size = 60
print(Data_size)
latent_space = 100

class Discriminator(nn.Module):
    
    def __init__(self):
        # initialise parent pytorch class
        super().__init__()
        
        # define neural network layers
        self.model = nn.Sequential(
            nn.Linear(Data_size, latent_space), # (Datasize,latent_space)
            nn.LeakyReLU(0.02),
            nn.LayerNorm(latent_space), 
            nn.Linear(latent_space, 1), # (latent_space ,1)
            nn.Sigmoid()
        )
        
        # create loss function
        self.loss_function = nn.BCELoss()

        # create optimiser, simple stochastic gradient descent
        self.optimiser = torch.optim.Adam(self.parameters(), lr=0.0001)

        # counter and accumulator for progress
        self.counter = 0;
        self.progress = []
        pass
    
    def forward(self, inputs):
        # simply run model
        return self.model(inputs)
    
    def train(self, inputs, targets):
        # calculate the output of the network
        outputs = self.forward(inputs)
        
        # calculate loss
        loss = self.loss_function(outputs, targets)

        # increase counter and accumulate error every 10
        self.counter += 1;
        if (self.counter % 10 == 0):
            self.progress.append(loss.item())
            pass
        if (self.counter % 3000 == 0):
            print("counter = ", self.counter)
            pass

        # zero gradients, perform a backward pass, update weights
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()
        pass
    
    def plot_progress(self):
        df = pd.DataFrame(self.progress, columns=['Discriminator loss'])
        df.plot(ylim=(0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5, 1.0, 5.0))
        pass
    
    pass

In [ ]:
def generate_random(size) :
  random_data = torch.rand(size)
  return random_data

def generate_random_seed(size):
    random_data = torch.randn(size)
    return random_data  

def generate_random_image(size) : 
  random_data = torch.rand(size)
  return random_data

In [ ]:
#Training the Discriminator
N = 1000  #iterations
#train_gan_flatten = train_gan_flatten.double
#print((train_gan_flatten.double)) #pandas.core.frame.DataFrame

D = Discriminator()
targets = torch.FloatTensor([1.0])
non_target = torch.FloatTensor([0.0])

#for i in range(N) : 
#  D.train(train_gan_flatten, targets)
#  D.train(generate_random_image(Data_size), non_target)

#  pass

In [ ]:
#Checking the performance of Discriminator
#D.plot_progress()

In [ ]:
# Constructing Generator

class Generator(nn.Module) : 
  def __init__(self) : 
    super().__init__()

    self.model = nn.Sequential(
        nn.Linear(100,200),
        nn.LeakyReLU(0.02),
        nn.LayerNorm(200),

        nn.Linear(200,Data_size),
        nn.Sigmoid() #과연 Sigmoid가 최선인가
    )

    self.optimiser = torch.optim.Adam(self.parameters(), lr = 0.0001)
    self.counter = 0
    self.progress = []

    pass

  def forward(self, inputs) : 
    return self.model(inputs)

  def train(self, D, inputs, targets) : 
    g_output = self.forward(inputs)
    d_output = D.forward(g_output)
    loss = D.forward(g_output)
    loss = D.loss_function(d_output, targets)

    self.counter+=1;
    if (self.counter % 10 == 0) :
      self.progress.append(loss.item())
      pass

    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()

  def plot_progress(self):
        df = pd.DataFrame(self.progress, columns=['Generator loss'])
        df.plot(ylim=(0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5, 1.0, 5.0))
        
  pass

In [ ]:
#Training both Generator and Discriminator for pattern
D = Discriminator()
G = Generator()

epochs = 1000
iteration = int(len(train_gan)/Data_size)*2 - 1
print("Data size : ", Data_size, "len(train_gan) : ", len(train_gan), "iteration : ",iteration)
for i in range(iteration) : #i는 0부터 시작 
  print(i)
  for epoch in range(epochs) :
    ref = train_gan[(i)*int(Data_size/2) : (i+2)*int(Data_size/2)] #Normalizing된 신호
    ref = torch.tensor(ref, dtype = torch.float32)

    D.train(ref, torch.FloatTensor([1.0]))
    D.train(G.forward(generate_random_seed(100)).detach(),torch.FloatTensor([0.0]))

    G.train(D, generate_random_seed(100), torch.FloatTensor([1.0]))
    pass
  pass


In [ ]:
#Plotting the Generator Loss
D.plot_progress()
G.plot_progress()
print(len(train_gan)) 

In [ ]:
#Test
seed = generate_random_seed(100)
out = G.forward(seed)
out_value = out.detach().numpy()

out_value_denorm = back_MinMax(out_value, train_gan_max, train_gan_min)
#print(out_value_denorm)
#print(out_value)

plt.figure()
plt.plot(out_value_denorm, label = "Generated Voltage Data")
plt.legend(prop = {'size':10})

train_gan_denorm = back_MinMax(train_gan, train_gan_max, train_gan_min)

plt.figure()
plt.plot(train_gan_denorm[0:Data_size], label = "Original Data")
plt.legend(prop = {'size':10})

In [ ]:
now = datetime.datetime.now()
print(now) 


In [ ]:
#Test2
seed2 = generate_random_seed(100)
out = G.forward(seed2)
out_value = out.detach().numpy()
out_value_denorm2 = back_MinMax(out_value, train_gan_max, train_gan_min)
#print(out_value_denorm)

#Test3
seed3 = generate_random_seed(100)
out = G.forward(seed3)
out_value = out.detach().numpy()
out_value_denorm3 = back_MinMax(out_value, train_gan_max, train_gan_min)

#Test4
seed4 = generate_random_seed(100)
out = G.forward(seed4)
out_value = out.detach().numpy()
out_value_denorm4 = back_MinMax(out_value, train_gan_max, train_gan_min)

plt.figure()
plt.plot(out_value_denorm2, label = "Generated Voltage Data1")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(out_value_denorm3, label = "Generated Voltage Data2")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(out_value_denorm4, label = "Generated Voltage Data3")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(train_gan_denorm[len(train_gan) - Data_size:len(train_gan)], label = "Original Data")
plt.legend(prop = {'size':10})

In [ ]:
#Test2
seed2 = generate_random_seed(100)
out = G.forward(seed2)
out_value = out.detach().numpy()
out_value_denorm2 = back_MinMax(out_value, train_gan_max, train_gan_min)
#print(out_value_denorm)

#Test3
seed3 = generate_random_seed(100)
out = G.forward(seed3)
out_value = out.detach().numpy()
out_value_denorm3 = back_MinMax(out_value, train_gan_max, train_gan_min)

#Test4
seed4 = generate_random_seed(100)
out = G.forward(seed4)
out_value = out.detach().numpy()
out_value_denorm4 = back_MinMax(out_value, train_gan_max, train_gan_min)

plt.figure()
plt.plot(out_value_denorm2, label = "Generated Voltage Data4")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(out_value_denorm3, label = "Generated Voltage Data5")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(out_value_denorm4, label = "Generated Voltage Data6")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(train_gan_denorm[len(train_gan) - Data_size:len(train_gan)], label = "Original Data")
plt.legend(prop = {'size':10})